### Final Project
### Topic - CSPNet implement in Super resolution
- Author
    - 109511054 王宏銘
    - 109511085 洪鈺翔


In [ ]:
import os
# Image
import numpy as np
from PIL import Image
import cv2
# Pytorch
import torch
import torch.nn as nn
import torchvision.transforms as tt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
# tqdm
from tqdm import tqdm
# other
import math
!pip install thop
import thop
# Tensor board
from torch.utils.tensorboard import SummaryWriter

##### DIV2K dataset
- Website : [link](https://data.vision.ee.ethz.ch/cvl/DIV2K/)

In [ ]:
if not os.path.isdir('dataset'):
    os.mkdir('dataset')
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip -O ./dataset/DIV2K_train_LR_bicubic_X4.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X4.zip -O ./dataset/DIV2K_valid_LR_bicubic_X4.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip -O ./dataset/DIV2K_train_HR.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip -O ./dataset/DIV2K_valid_HR.zip

--2023-12-11 12:35:16--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip [following]
--2023-12-11 12:35:16--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246914039 (235M) [application/zip]
Saving to: ‘./dataset/DIV2K_train_LR_bicubic_X4.zip’

./dataset/DIV2K_tra 100%[===================>] 235.47M   222MB/s    in 1.1s    

2023-12-11 12:35:18 (222 MB/s) - ‘./dataset/DIV2K_train_LR_bicubic_X4.zip’ saved [246914039/246914039]

--2023-12-11 12:35:18--  http://data.vision.e

In [ ]:
!unzip ./dataset/DIV2K_train_LR_bicubic_X4.zip -d ./dataset/
!unzip ./dataset/DIV2K_valid_LR_bicubic_X4.zip -d ./dataset/
!unzip ./dataset/DIV2K_train_HR.zip -d ./dataset/
!unzip ./dataset/DIV2K_valid_HR.zip -d ./dataset/

Archive:  ./dataset/DIV2K_train_LR_bicubic_X4.zip
   creating: ./dataset/DIV2K_train_LR_bicubic/X4/
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0782x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0119x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0625x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0072x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0350x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0691x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0070x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0569x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0733x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0790x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0515x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0218x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0800x4.png  
  inflating: ./dataset/DIV2K_train_LR_bicubic/X4/0478x4.png  
  inflating: ./dataset/DIV2K_tra

In [ ]:
!rm ./dataset/DIV2K_train_LR_bicubic_X4.zip
!rm ./dataset/DIV2K_valid_LR_bicubic_X4.zip
!rm ./dataset/DIV2K_train_HR.zip
!rm ./dataset/DIV2K_valid_HR.zip

##### Preprocess

In [ ]:
train_x_dir = './dataset/DIV2K_train_LR_bicubic/X4/'
train_y_dir = './dataset/DIV2K_train_HR/'
val_x_dir  = './dataset/DIV2K_valid_LR_bicubic/X4/'
val_y_dir  = './dataset/DIV2K_valid_HR/'
crop_tx_dir = './dataset/DIV2K_train_LR_bicubic/X4_crop_25/'
crop_ty_dir = './dataset/DIV2K_train_HR_crop_100/'
crop_vx_dir = './dataset/DIV2K_valid_LR_bicubic/X4_crop_200/'
crop_vy_dir = './dataset/DIV2K_valid_HR_crop_800/'

# Crop
def CROP(original_dir:str, crop_dir:str, crop_size=25):

    if os.path.isdir(crop_dir):
        return

    os.mkdir(crop_dir)
    M = crop_size
    N = crop_size

    for dirpath, dirnames, filenames in os.walk(original_dir):
        for filename in tqdm(filenames):
            img = cv2.imread(original_dir+filename)
            s = np.shape(img)
            image_copy = img
            imgheight, imgwidth, _ = np.shape(img)
            x1 = 0
            y1 = 0
            for y in range(0, imgheight, M):
                for x in range(0, imgwidth, N):
                    if (imgheight - y) < M or (imgwidth - x) < N:
                        break
                    y1 = y + M
                    x1 = x + N
                    # check whether the patch width or height exceeds the image width or height
                    if x1 >= imgwidth and y1 >= imgheight:
                        x1 = imgwidth - 1
                        y1 = imgheight - 1
                        #Crop into patches of size MxN
                        tiles = image_copy[y:y+M, x:x+N]
                        #Save each patch into file directory
                        cv2.imwrite(crop_dir+str(int(x/N))+'_'+str(int(y/M))+filename, tiles)
                    elif y1 >= imgheight: # when patch height exceeds the image height
                        y1 = imgheight - 1
                        #Crop into patches of size MxN
                        tiles = image_copy[y:y+M, x:x+N]
                        #Save each patch into file directory
                        cv2.imwrite(crop_dir+str(int(x/N))+'_'+str(int(y/M))+filename, tiles)
                    elif x1 >= imgwidth: # when patch width exceeds the image width
                        x1 = imgwidth - 1
                        #Crop into patches of size MxN
                        tiles = image_copy[y:y+M, x:x+N]
                        #Save each patch into file directory
                        cv2.imwrite(crop_dir+str(int(x/N))+'_'+str(int(y/M))+filename, tiles)
                    else:
                        #Crop into patches of size MxN
                        tiles = image_copy[y:y+M, x:x+N]
                        #Save each patch into file directory
                        cv2.imwrite(crop_dir+str(int(x/N))+'_'+str(int(y/M))+filename, tiles)
    return

def Remove_Plain_Color(lr_dir:str, hr_dir:str):

    for root, dirs, files in os.walk(lr_dir, topdown=True):
        lr_img_names = files
    for root, dirs, files in os.walk(hr_dir, topdown=True):
        hr_img_names = files

    lr_img_names.sort()
    hr_img_names.sort()

    for lr_file, hr_file in zip(lr_img_names, hr_img_names):

        lr_img = cv2.imread(lr_dir+lr_file)/255.
        hr_img = cv2.imread(hr_dir+hr_file)/255.

        lr_img = torch.asarray(lr_img).reshape(-1,lr_img.shape[2],lr_img.shape[0],lr_img.shape[1])
        hr_img = torch.asarray(hr_img).reshape(-1,hr_img.shape[2],hr_img.shape[0],hr_img.shape[1])

        lr_up = nn.functional.interpolate(lr_img, scale_factor=4, mode='bicubic')

        mse = np.square(lr_up.numpy() - hr_img.numpy()).mean()

        if mse < 1e-8:
            os.remove(lr_dir+lr_file)
            os.remove(hr_dir+hr_file)

CROP(original_dir=train_x_dir, crop_dir=crop_tx_dir, crop_size=25)
CROP(original_dir=train_y_dir, crop_dir=crop_ty_dir, crop_size=100)
CROP(original_dir=val_x_dir, crop_dir=crop_vx_dir, crop_size=200)
CROP(original_dir=val_y_dir, crop_dir=crop_vy_dir, crop_size=800)
Remove_Plain_Color(crop_tx_dir, crop_ty_dir)
#Remove_Plain_Color(crop_vx_dir, crop_vy_dir)

100%|██████████| 100/100 [00:15<00:00,  6.33it/s]


##### Fix seed

In [ ]:
def set_seed(seed):
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(123)

##### Utils

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

#------------------------------------------------------------------------------------------------------------------------
#   PSNR measure function
#       - original code
#           + Website :         https://data.vision.ee.ethz.ch/cvl/DIV2K/
#           + Downloaded from : https://competitions.codalab.org/my/datasets/download/1ed1e509-86ed-4dc1-be3f-4bca1dbdbd83
#------------------------------------------------------------------------------------------------------------------------
def output_measures(img_orig:torch.Tensor, img_out:torch.Tensor):
    img_orig = img_orig.numpy()
    img_out = img_out.numpy()
    assert (img_orig.shape == img_out.shape), 'shape of images should be the same.'
    batch_size = img_orig.shape[0]
    psnr_sum = 0.
    for b in range(batch_size):
        squared_error = np.square(img_orig[b] - img_out[b])
        mse = np.mean(squared_error)
        psnr = 10 * np.log10(1.0 / mse)
        psnr_sum += psnr
    # Return sum of PSNR
    return psnr_sum


##### Dataset

In [ ]:
class ISR_Dataset(Dataset):
    def __init__(self, x_path, y_path, x_transform, y_transform):
        # x_path : low  res img
        # y_path : high res img
        self.x_path = x_path
        self.y_path = y_path
        self.x_transform = x_transform
        self.y_transform = y_transform
        # Load image names
        for root, dirs, files in os.walk(x_path, topdown=True):
            self.x_img_names = files
        for root, dirs, files in os.walk(y_path, topdown=True):
            self.y_img_names = files
        self.x_img_names.sort()
        self.y_img_names.sort()

        #print(self.x_img_names)
        #print(self.y_img_names)
        print('len_x : {}'.format(self.x_img_names.__len__()))
        print('len_y : {}'.format(self.y_img_names.__len__()))
        assert (self.x_img_names.__len__() == self.y_img_names.__len__()), 'The target length should be same as the input length'

        # Save data length
        self.data_len = self.x_img_names.__len__()

    def __getitem__(self, index):
        # Read image
        x_img = Image.open(self.x_path + self.x_img_names[index])
        y_img = Image.open(self.y_path + self.y_img_names[index])
        # Transformation
        x_img = x_img.convert('YCbCr').getchannel('Y') # Y channel
        y_img = y_img.convert('YCbCr').getchannel('Y') # Y channel
        x_img = self.x_transform(x_img)
        y_img = self.y_transform(y_img)

        return x_img, y_img

    def __len__(self):
        return self.data_len

In [ ]:
transform = tt.ToTensor()
# Dataset
train_ds = ISR_Dataset(x_path=crop_tx_dir, y_path=crop_ty_dir, x_transform=transform, y_transform=transform)
valid_ds = ISR_Dataset(x_path=crop_vx_dir, y_path=crop_vy_dir, x_transform=transform, y_transform=transform)
# Dataloader
batch_size = 32
batch_size_val = 2
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=max(os.cpu_count()-2,2), pin_memory=False)
valid_dl = DataLoader(valid_ds, batch_size=batch_size_val, shuffle=False, num_workers=max(os.cpu_count()-2,2), pin_memory=False)


len_x : 209535
len_y : 209535
len_x : 210
len_y : 210


##### Model

In [ ]:
#=============================================================================
#   Source
#       -  https://github.com/yjn870/SRDenseNet-pytorch/blob/master/models.py
#=============================================================================
import torch.nn as nn
class ConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(ConvLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.conv(x))


class DenseLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(DenseLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return torch.cat([x, self.relu(self.conv(x))], 1)


class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers):
        super(DenseBlock, self).__init__()
        self.block = [ConvLayer(in_channels, growth_rate, kernel_size=3)]
        for i in range(num_layers - 1):
            self.block.append(DenseLayer(growth_rate * (i + 1), growth_rate, kernel_size=3))
        self.block = nn.Sequential(*self.block)

    def forward(self, x):
        return torch.cat([x, self.block(x)], 1)


class SRDenseNet(nn.Module):
    def __init__(self, num_channels=1, growth_rate=16, num_blocks=8, num_layers=8):
        super(SRDenseNet, self).__init__()

        # low level features
        self.conv = ConvLayer(num_channels, growth_rate * num_layers, 3)

        # high level features
        self.dense_blocks = []
        for i in range(num_blocks):
            self.dense_blocks.append(DenseBlock(growth_rate * num_layers * (i + 1), growth_rate, num_layers))
        self.dense_blocks = nn.Sequential(*self.dense_blocks)

        # bottleneck layer
        self.bottleneck = nn.Sequential(
            nn.Conv2d(growth_rate * num_layers + growth_rate * num_layers * num_blocks, 256, kernel_size=1),
            nn.ReLU(inplace=True)
        )

        # deconvolution layers
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(256, 256, kernel_size=3, stride=2, padding=3 // 2, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 256, kernel_size=3, stride=2, padding=3 // 2, output_padding=1),
            nn.ReLU(inplace=True)
        )

        # reconstruction layer
        self.reconstruction = nn.Conv2d(256, num_channels, kernel_size=3, padding=3 // 2)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                nn.init.kaiming_normal_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias.data)

    def forward(self, x):
        x = self.conv(x)
        x = self.dense_blocks(x)
        x = self.bottleneck(x)
        x = self.deconv(x)
        x = self.reconstruction(x)
        return x
#=====================================================================
# below is self-implement DenseNet
# reference https://github.com/yjn870/SRDenseNet-pytorch/blob/master/models.py
# reference https://github.com/Lornatang/SRDenseNet-PyTorch/blob/master/model.py
#=====================================================================
class denselayer(nn.Module):
  def __init__(self, num_input_feature, out_channel):
    super(denselayer, self).__init__()
    #self.norm1 = nn.BatchNorm2d(num_input_feature)
    self.relu1 = nn.ReLU(inplace=True)
    self.conv1 = nn.Conv2d(num_input_feature, out_channel, kernel_size=3, stride=1, padding = 1)
  def forward(self, x):
    out = torch.cat([x, self.relu1(self.conv1(x))], 1)
    return out
class denseblock(nn.Module):
  def __init__(self, num_input_feature, growth_rate, num_layer):
    super(denseblock, self).__init__()
    self.layers = nn.Sequential(
        nn.Conv2d(num_input_feature, growth_rate, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(True)
    )
    for i in range(num_layer - 1):
      layer = denselayer((i + 1) * growth_rate, growth_rate )
      self.layers.add_module("denselayer%d" % (i + 1), layer)
  def forward(self, in_feature):
    return torch.cat([in_feature, self.layers(in_feature)], 1)
class CSP_block(nn.Module):
  def __init__(self, num_input_feature, growth_rate, num_layer, partial_ratio = 0.5):
    super(CSP_block, self).__init__()
    self.partial_ch1 = int(num_input_feature * partial_ratio)
    self.partial_ch2 = num_input_feature - self.partial_ch1
    self.dense = denseblock(self.partial_ch2, growth_rate, num_layer)
    trans_ch = self.partial_ch2 + growth_rate * num_layer
    self.conv1 = nn.Conv2d(trans_ch, trans_ch, kernel_size = 1, stride = 1, padding = 0)
  def forward(self, x):
    part1 = x[:, :self.partial_ch1, :, :]
    part2 = x[:, :self.partial_ch2, :, :]
    part2 = self.dense(part2)
    part2 = self.conv1(part2)
    out = torch.cat((part1,part2), 1)
    return out
#in paper we need 8 denseblock and each block has 8 conv layer
# I cancel the first conv layer and the classifier
class DenseNet(nn.Module):
  def __init__(self, num_channel, growth_rate = 4, num_block = 2, num_layer = 2):
    super(DenseNet, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(num_channel, growth_rate * num_layer, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
    )
    self.dense_block = nn.Sequential()
    for i in range(num_block):
      self.dense_block.add_module("denseblock%d" % (i + 1) ,denseblock(growth_rate * num_layer * (i + 1), growth_rate, num_layer))
    self.BottleNeck = nn.Sequential(
        nn.Conv2d(growth_rate * num_layer + growth_rate * num_layer * num_block, 256, kernel_size = 1),
        nn.ReLU(inplace = True)
    )
    self.deconv1 = nn.ConvTranspose2d(256, 256, kernel_size = 3, stride = 2, padding = 1, output_padding = 1)
    self.relu1 = nn.ReLU(True)
    self.deconv2 = nn.ConvTranspose2d(256, 256, kernel_size = 3, stride = 2, padding = 1, output_padding = 1)
    self.relu2 = nn.ReLU(True)
    self.conv1 = nn.Conv2d(256, num_channel, kernel_size = 3, stride = 1, padding = 1)
    self._initialize_weights()
  def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                nn.init.kaiming_normal_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias.data)
  def forward(self, x):
    x = self.features(x)
    x = self.dense_block(x)
    x = self.BottleNeck(x)
    x = self.deconv1(x)
    x = self.relu1(x)
    x = self.deconv2(x)
    x = self.relu2(x)
    x = self.conv1(x)
    return x
class SRCSPNet(nn.Module):
  def __init__(self, num_channel, growth_rate = 4, num_block = 8, num_layer = 8):
    super(SRCSPNet, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(num_channel, growth_rate * num_layer, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
    )
    self.dense_block = nn.Sequential()

    self.dense_block.add_module("CSP_block1" ,CSP_block(128, growth_rate, num_layer))
    self.dense_block.add_module("CSP_block2" ,CSP_block(192, growth_rate, num_layer))
    self.dense_block.add_module("CSP_block3" ,CSP_block(224, growth_rate, num_layer))
    self.dense_block.add_module("CSP_block4" ,CSP_block(240, growth_rate, num_layer))
    self.dense_block.add_module("CSP_block5" ,CSP_block(248, growth_rate, num_layer))
    self.dense_block.add_module("CSP_block6" ,CSP_block(252, growth_rate, num_layer))
    self.dense_block.add_module("CSP_block7" ,CSP_block(254, growth_rate, num_layer))
    self.dense_block.add_module("CSP_block8" ,CSP_block(255, growth_rate, num_layer))
    self.BottleNeck = nn.Sequential(
        nn.Conv2d(383, 256, kernel_size = 1),
        nn.ReLU(inplace = True)
    )
    self.upsample = nn.Upsample(scale_factor = 4)
    self.deconv1 = nn.ConvTranspose2d(128, 128, kernel_size = 3, stride = 2, padding = 1, output_padding = 1)
    self.relu1 = nn.ReLU(True)
    self.deconv2 = nn.ConvTranspose2d(128, 128, kernel_size = 3, stride = 2, padding = 1, output_padding = 1)
    self.relu2 = nn.ReLU(True)
    self.conv1 = nn.Conv2d(256, num_channel, kernel_size = 3, stride = 1, padding = 1)
    self._initialize_weights()
  def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                nn.init.kaiming_normal_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias.data)
  def forward(self, x):
    x = self.features(x)
    x = self.dense_block(x)
    x = self.BottleNeck(x)
    x1 = x[:, :128, :, :]
    x2 = x[:, :128, :, :]
    x1 = self.deconv1(x1)
    x1 = self.relu1(x1)
    x1 = self.deconv2(x1)
    x1 = self.relu2(x1)
    x2 = self.upsample(x2)
    x = torch.cat((x1, x2),1)
    x = self.conv1(x)
    return x

class Up_sample_model(nn.Module):
    def __init__(self):
        super(Up_sample_model, self).__init__()
        self.dummy = nn.Linear(1,1)

    def forward(self, x):
        return nn.functional.interpolate(x, scale_factor=4, mode='bicubic')


##### Arguments and Settings

In [ ]:
args = {
    'lr': 1e-4,
    'epochs': 10,
    'save_dir': './save/srdense_11_30_0/',
    'project_name': 'test_0',
    'train': True,
    'tensorboard': True,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
}

#model = Up_sample_model()
model = DenseNet(num_channel=1, growth_rate=16, num_block=8, num_layer=8).to(args['device'])
#model = SRCSPNet(num_channel = 1, growth_rate = 16, num_block = 8, num_layer = 8).to(args['device'])
#print(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=args['lr'], betas=(0.9,0.999), eps=1e-8, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=args['epochs'], eta_min=1e-7)
criterion = nn.L1Loss()
writer = SummaryWriter(log_dir=args['save_dir']+'tensorboard/', filename_suffix='_'+args['project_name'])



colab tensorboard

In [ ]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(args['save_dir']+'tensorboard/')
)
# Install
! npm install -g localtunnel

# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

# Get url
! cat url.txt
! curl ipv4.icanhazip.com

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.392s
your url is: https://cold-rivers-guess.loca.lt
your url is: https://full-swans-speak.loca.lt
your url is: https://fifty-lions-rhyme.loca.lt
/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:37039 (check your firewall settings)
    at Socket.<anonymous> (/tools/node/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (events.js:315:20)
    at emitErrorNT (internal/streams/destroy.js:106:8)
    at emitErrorCloseNT (internal/streams/destroy.js:74:3)
    at processTicksAndRejections (internal/process/task_queues.js:80:21)
your url is: https://tricky-tigers-speak.loca.lt
your url is: https://orange-berries-lose.loca.lt
/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:39037 (check your firewall settings)
    at Soc

##### Profiling model

In [ ]:
input = torch.rand(1,1,256,256).to(args['device'])
print(input.size())
flops, params = thop.profile(model, inputs=[input])
print('FLOPs = ' + str(flops/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')

torch.Size([1, 1, 256, 256])
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.ConvTranspose2d'>.
FLOPs = 872.222294016G
Params = 2.659585M


In [ ]:
# Tensor board
data_it = iter(valid_dl)
img_x, img_y = next(data_it)
writer.add_graph(model, img_x.to(args['device']))
data_it = iter(valid_ds)
img_x, img_y = next(data_it)
writer.add_image('Low res image', img_x)
writer.add_image('High res image', img_y)

##### Training

In [ ]:
print('Device using : {}'.format(args['device']))

# Create folder
if not os.path.isdir('save'):
    os.mkdir('save')
if not os.path.isdir(args['save_dir']):
    os.mkdir(args['save_dir'])

best_val_loss = math.inf

# Training Loop
for epoch in range(args['epochs']):
    print('[Epoch {}]'.format(epoch))

    # Training
    if args['train'] == True:
        print('Train')
        model.train()
        loss_sum = 0.
        for data in tqdm(train_dl):
            img_out = model(data[0].to(args['device']))
            loss = criterion(img_out, data[1].to(args['device']))
            loss_sum += loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = loss_sum/len(train_dl)
        lr = get_lr(optimizer)
        print('avg_loss = {:e}'.format(avg_loss), end=' ')
        print('learning_rate = {:.6f}'.format(lr))

        if args['tensorboard']:
            writer.add_scalar('Loss/Train', avg_loss, epoch)
            writer.add_scalar('Learning rate', lr, epoch)

    # Validating
    print('Valid')
    model.eval()
    loss_sum = 0.
    psnr_sum = 0.
    img_tb_ready = False

    with torch.no_grad():
        for data in tqdm(valid_dl):
            img_out = model(data[0].to(args['device']))
            loss = criterion(img_out, data[1].to(args['device']))
            loss_sum += loss
            psnr_sum += output_measures(img_out.cpu(), data[1])

            if not img_tb_ready and args['tensorboard']:
                writer.add_image('Output_img', img_out[0], global_step=epoch)
                img_tb_ready = True

    avg_loss = loss_sum/len(valid_dl)
    psnr = psnr_sum/valid_ds.__len__()

    print('avg_loss = {:e}'.format(avg_loss))
    print('PSNR     = {:.2f} dB'.format(psnr))

    if args['tensorboard']:
        writer.add_scalar('Loss/Valid', avg_loss, epoch)
        writer.add_scalar('PSNR/Valid', psnr, epoch)

    # Save model
    torch.save(model.state_dict(), args['save_dir']+'model_save.pth')

    # Save best models
    if loss_sum/valid_ds.__len__() < best_val_loss:
        best_val_loss = loss_sum/len(valid_dl)
        torch.save(model.state_dict(), args['save_dir']+'best_model_save.pth')

    if args['train'] == False:
        break

    # Scheduler
    scheduler.step()

writer.close()

Device using : cuda
[Epoch 0]
Train


100%|██████████| 6548/6548 [06:23<00:00, 17.08it/s]


avg_loss = 4.932886e-02 learning_rate = 0.000100
Valid


100%|██████████| 105/105 [00:08<00:00, 12.15it/s]


avg_loss = 2.673650e-02
PSNR     = 28.20 dB
[Epoch 1]
Train


...

avg_loss = 2.459310e-02
PSNR     = 28.87 dB
[Epoch 9]
Train


100%|██████████| 6548/6548 [06:25<00:00, 17.00it/s]


avg_loss = 4.450976e-02 learning_rate = 0.000003
Valid


100%|██████████| 105/105 [00:08<00:00, 12.14it/s]

avg_loss = 2.456417e-02
PSNR     = 28.89 dB


upsample

In [ ]:
writer = SummaryWriter(log_dir=args['save_dir']+'tensorboard/', filename_suffix='_'+args['project_name'])
model = nn.Upsample(scale_factor=4)
img_tb_ready = False
for data in tqdm(valid_dl):
  img_out = model(data[0].to(args['device']))
  writer.add_image('upsample result', img_out[0])
  if not img_tb_ready and args['tensorboard']:
                writer.add_image('upsample_img', img_out[0], global_step=epoch)
                img_tb_ready = True
writer.close()



100%|██████████| 105/105 [00:06<00:00, 16.42it/s]


##### Bicubic upsample : 27.22 dB